In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 학습 데이터와 테스트 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
test_file_path = '/content/test.csv'      # 테스트 데이터 파일 경로
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)
test_data.fillna('WT', inplace=True)

# 특징과 타겟 분리
X_train = train_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y_train = train_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 첫 글자 + 컬럼명 조합으로 변환
def encode_feature(col):
    return col.apply(lambda x: '' if x == 'WT' else f"{x[0]}_{col.name}")

# 모든 특성에 대해 인코딩 적용
X_train_encoded = X_train.apply(encode_feature)
X_test_encoded = test_data.drop(columns=['ID']).apply(encode_feature)  # 테스트 데이터 인코딩

# 레이블 인코딩
le = LabelEncoder()
for c in X_train_encoded.columns:
    X_train_encoded[c] = le.fit_transform(X_train_encoded[c])

# 테스트 데이터에도 동일한 인코딩 적용
for c in X_test_encoded.columns:
    if c in le.classes_:
        X_test_encoded[c] = le.transform(X_test_encoded[c])
    else:
        X_test_encoded[c] = le.fit_transform(X_test_encoded[c])

# 레이블 인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y_train)

# 훈련 데이터와 검증 데이터 분리
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_encoded, y_encoded, test_size=0.2, random_state=42)

# NumPy 배열로 변환
X_train_final = X_train_final.values
X_val = X_val.values
X_test_encoded = X_test_encoded.values

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_final.shape[1],)),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le_y.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
model.fit(X_train_final, y_train_final, validation_data=(X_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[early_stopping])

# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# 테스트 데이터 예측
test_predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_labels = le_y.inverse_transform(test_predictions.argmax(axis=1))

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions_combined_data.csv', index=False)

print("Predictions saved to 'predictions_combined_data.csv'.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.1269 - loss: 3.1852 - val_accuracy: 0.2627 - val_loss: 2.6081
Epoch 2/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.3300 - loss: 2.3076 - val_accuracy: 0.2877 - val_loss: 2.3395
Epoch 3/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5059 - loss: 1.5872 - val_accuracy: 0.3022 - val_loss: 2.2999
Epoch 4/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.6572 - loss: 1.0505 - val_accuracy: 0.3038 - val_loss: 2.4230
Epoch 5/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7403 - loss: 0.7671 - val_accuracy: 0.3046 - val_loss: 2.6603
Epoch 6/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8010 - loss: 0.5951 - val_accuracy: 0.2933 - val_loss: 2.8113
Epoch 7/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.8323 - loss: 0.5183 - val_accuracy: 0.2965 - val_loss: 2.9689
Epoch 8/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8594 - loss: 0.4128 - 

In [10]:
results.SUBCLASS.value_counts()

,count
SUBCLASS,
STES,745
BRCA,503
KIPAN,204
COAD,153
THCA,123
GBMLGG,104
SKCM,93
PRAD,93
LGG,90


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 학습 데이터와 테스트 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
test_file_path = '/content/test.csv'      # 테스트 데이터 파일 경로
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)
test_data.fillna('WT', inplace=True)

# 특징과 타겟 분리
X_train = train_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y_train = train_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 첫 글자 + 컬럼명 조합으로 변환
def encode_feature(col):
    return col.apply(lambda x: '' if x == 'WT' else f"{x[0]}_{col.name}")

# 모든 특성에 대해 인코딩 적용
X_train_encoded = X_train.apply(encode_feature)
X_test_encoded = test_data.drop(columns=['ID']).apply(encode_feature)  # 테스트 데이터 인코딩

# 레이블 인코딩
le = LabelEncoder()
for c in X_train_encoded.columns:
    X_train_encoded[c] = le.fit_transform(X_train_encoded[c])

# 테스트 데이터에도 동일한 인코딩 적용
for c in X_test_encoded.columns:
    if c in le.classes_:
        X_test_encoded[c] = le.transform(X_test_encoded[c])
    else:
        X_test_encoded[c] = le.fit_transform(X_test_encoded[c])

# 레이블 인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y_train)

# 훈련 데이터와 검증 데이터 분리
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_encoded, y_encoded, test_size=0.2, random_state=42)

# NumPy 배열로 변환
X_train_final = X_train_final.values
X_val = X_val.values
X_test_encoded = X_test_encoded.values

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_final.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le_y.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정

])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
model.fit(X_train_final, y_train_final, validation_data=(X_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[early_stopping])

# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# 테스트 데이터 예측
test_predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_labels = le_y.inverse_transform(test_predictions.argmax(axis=1))

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('p00001redictions_combined_data.csv', index=False)

print("Predictions saved to 'predictions_combined_data.csv'.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.0462 - loss: 3.6910 - val_accuracy: 0.0967 - val_loss: 3.1521
Epoch 2/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.0941 - loss: 3.2340 - val_accuracy: 0.1378 - val_loss: 3.0817
Epoch 3/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.1266 - loss: 3.0573 - val_accuracy: 0.1821 - val_loss: 3.0262
Epoch 4/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.1700 - loss: 2.9229 - val_accuracy: 0.2095 - val_loss: 2.9457
Epoch 5/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.2223 - loss: 2.8218 - val_accuracy: 0.2385 - val_loss: 2.8510
Epoch 6/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.2482 - loss: 2.6875 - val_accuracy: 0.2425 - val_loss: 2.7435
Epoch 7/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.2966 - loss: 2.5203 - val_accuracy: 0.2434 - val_loss: 2.6473
Epoch 8/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.3156 - loss: 2.3662 - 

In [14]:
results.SUBCLASS.value_counts()


,count
SUBCLASS,
BRCA,641
COAD,392
STES,350
KIPAN,249
GBMLGG,128
KIRC,113
SKCM,107
THCA,81
LGG,62


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.0570 - loss: 3.6941 - val_accuracy: 0.1297 - val_loss: 3.1535 - learning_rate: 1.0000e-04
Epoch 2/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.1060 - loss: 3.2143 - val_accuracy: 0.1636 - val_loss: 3.0857 - learning_rate: 1.0000e-04
Epoch 3/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.1396 - loss: 3.0595 - val_accuracy: 0.1821 - val_loss: 3.0273 - learning_rate: 1.0000e-04
Epoch 4/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.1693 - loss: 2.9207 - val_accuracy: 0.2168 - val_loss: 2.9476 - learning_rate: 1.0000e-04
Epoch 5/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.2216 - loss: 2.8005 - val_accuracy: 0.2248 - val_loss: 2.8488 - learning_rate: 1.0000e-04
Epoch 6/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.2646 - loss: 2.6403 - val_accuracy: 0.2409 - val_loss: 2.7296 - learning_rate: 1.0000e-04
Epoch 7/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/ste

In [16]:
results.SUBCLASS.value_counts()

,count
SUBCLASS,
BRCA,609
COAD,412
KIPAN,306
STES,251
GBMLGG,143
OV,118
UCEC,114
SKCM,101
THCA,90


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 로드 및 전처리
train_file_path = '/content/train.csv'
train_data = pd.read_csv(train_file_path)
train_data.fillna('WT', inplace=True)

X = train_data.drop(columns=['ID', 'SUBCLASS'])
y = train_data['SUBCLASS']

# 인코딩
def encode_feature(col):
    return col.apply(lambda x: '' if x == 'WT' else f"{x[0]}_{col.name}")

X_encoded = X.apply(encode_feature)
le = LabelEncoder()
for c in X_encoded.columns:
    X_encoded[c] = le.fit_transform(X_encoded[c])

le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)

# 학습 및 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# 모델 구성 (더 복잡한 구조)
model = keras.Sequential([
    layers.Dense(512, activation='leaky_relu', input_shape=(X_train.shape[1],)),  # Leaky ReLU 사용
    layers.Dropout(0.5),
    layers.Dense(256, activation='leaky_relu'),  # Leaky ReLU 사용
    layers.Dropout(0.5),
    layers.Dense(128, activation='leaky_relu'),  # Leaky ReLU 사용
    layers.Dense(len(le_y.classes_), activation='softmax')  # 소프트맥스 활성화
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[early_stopping])

# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.1350 - loss: 3.4621 - val_accuracy: 0.2804 - val_loss: 2.4879
Epoch 2/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.3871 - loss: 2.0746 - val_accuracy: 0.3006 - val_loss: 2.4108
Epoch 3/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.5169 - loss: 1.5775 - val_accuracy: 0.3038 - val_loss: 2.3729
Epoch 4/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.6438 - loss: 1.1354 - val_accuracy: 0.3086 - val_loss: 2.6019
Epoch 5/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.7133 - loss: 0.9385 - val_accuracy: 0.3086 - val_loss: 3.0874
Epoch 6/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.7679 - loss: 0.7019 - val_accuracy: 0.2990 - val_loss: 3.1983
Epoch 7/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8208 - loss: 0.5362 - val_accuracy: 0.3006 - val_loss: 3.6485
Epoch 8/200
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.8146 - loss: 0.5323 - 

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 학습 데이터와 테스트 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
test_file_path = '/content/test.csv'      # 테스트 데이터 파일 경로
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)
test_data.fillna('WT', inplace=True)

test_data['SUBCLASS'] = None
# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
#X_train = train_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
#y_train = train_data['SUBCLASS']
# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 첫 글자 + 컬럼명 조합으로 변환
def encode_feature(col):
    return col.apply(lambda x: '' if x == 'WT' else f"{str(x)[0]}_{col.name}")

# 모든 특성에 대해 인코딩 적용
X_encoded = X_encoded.apply(encode_feature)

# 레이블 인코딩
le = LabelEncoder()
for c in X_encoded.columns:
    X_encoded[c] = le.fit_transform(X_encoded[c])


# 레이블 인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y_train)



# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)
y_encoded_full.iloc[:len(y_encoded)] = y_encoded

# 학습/검증 데이터 분리
X_train_final, X_val, y_train_final, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)



# 훈련 데이터와 검증 데이터 분리
#X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_encoded, y_encoded, test_size=0.2, random_state=42)

# NumPy 배열로 변환
X_train_final = X_train_final.values
X_val = X_val.values

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_final.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le_y.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정

])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
model.fit(X_train_final, y_train_final, validation_data=(X_val, y_val),
          epochs=200, batch_size=32,
          callbacks=[early_stopping])

# 검증 데이터 평가
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# 테스트 데이터 예측

# 테스트 데이터 예측
X_test_encoded = X_encoded[len(y_encoded):]
predictions = model.predict(X_test_encoded)

#test_predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_classes = predictions.argmax(axis=1)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.0773 - loss: 3.2765 - val_accuracy: 0.1351 - val_loss: 3.1553
Epoch 2/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.1372 - loss: 3.0655 - val_accuracy: 0.1280 - val_loss: 3.1252
Epoch 3/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.1868 - loss: 2.9129 - val_accuracy: 0.1139 - val_loss: 3.1216
Epoch 4/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.2332 - loss: 2.6815 - val_accuracy: 0.1139 - val_loss: 3.1747
Epoch 5/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.3536 - loss: 2.3018 - val_accuracy: 0.1018 - val_loss: 3.3453
Epoch 6/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.4680 - loss: 1.9248 - val_accuracy: 0.0847 - val_loss: 3.6185
Epoch 7/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.5686 - loss: 1.6077 - val_accuracy: 0.0786 - val_loss: 3.9390
Epoch 8/200
124/124 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6607 - loss: 1.2901 - 

ValueError: y contains previously unseen labels: [10 11 15 19 20 21 23 25]

In [23]:

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le_y.inverse_transform(predicted_classes)


# 예측 결과 변환
#predicted_labels = le_y.inverse_transform(test_predictions.argmax(axis=1))

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('combined_p00001redictions_combined_data.csv', index=False)

print("Predictions saved to 'predictions_combined_data.csv'.")

ValueError: array length 1241 does not match index length 2546

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE  # SMOTE 사용을 위한 라이브러리 추가

# 학습 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '/content/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),  # 드롭아웃 비율 조정
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),  # 드롭아웃 비율 조정
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 테스트 데이터 예측
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분

predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('maybe_predictions_no_scaler.csv', index=False)

print("Predictions saved to 'predictions_no_scaler.csv'.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.1679 - loss: 2.9185 - val_accuracy: 0.2981 - val_loss: 2.3803
Epoch 2/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.5024 - loss: 1.5784 - val_accuracy: 0.3521 - val_loss: 2.1146
Epoch 3/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.7476 - loss: 0.7406 - val_accuracy: 0.3288 - val_loss: 2.4988
Epoch 4/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.8422 - loss: 0.4149 - val_accuracy: 0.3175 - val_loss: 2.8838
Epoch 5/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8870 - loss: 0.3416 - val_accuracy: 0.3151 - val_loss: 2.9985
Epoch 6/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.8968 - loss: 0.2448 - val_accuracy: 0.3159 - val_loss: 3.3755
Epoch 7/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9113 - loss: 0.2029 - val_accuracy: 0.2917 - val_loss: 3.5270
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Predictions saved to 'predictions_no_scaler.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE  # SMOTE 사용을 위한 라이브러리 추가

# 학습 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '/content/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),  # 드롭아웃 비율 조정
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),  # 드롭아웃 비율 조정
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 테스트 데이터 예측
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분

predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('maybe_predictions_no_scaler.csv', index=False)

print("Predictions saved to 'predictions_no_scaler.csv'.")
